In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
import xgboost as xgb

Read data, split data, standard scaler, encoding, column transfomer, pre-processer, transformation of x column by fit_tranform, train test split 80 20, evalute model (t,p), use mse mae rmc r2, use liner reg, lasso reg, regg, random forest, desion tree, xg boost,cat boost,knn, evalute train and test data set, model performace  for trainig and test, create a data frame of model name mse mae rmc r2 adjucted r2 like columns , select top 2 best models for prediction

import all the necessary libs mse, r2, mae, knn, decisiontree reg, randomforest reg, ada boost red, svr, linear reg, lasso, rig reg, randomsized search cv, catboost, xgboost


# Task
Analyze the provided dataset by training and evaluating multiple regression models, including Linear Regression, Lasso, Ridge, Random Forest, Decision Tree, XGBoost, CatBoost, and KNN. Preprocess the data using standard scaling and one-hot encoding, split it into training and testing sets (80/20), and evaluate each model using MSE, MAE, RMSE, and R2. Summarize the performance of all models in a DataFrame and identify the top 2 best-performing models.

## Load data

### Subtask:
Load the dataset into a pandas DataFrame.


**Reasoning**:
Load the dataset into a pandas DataFrame as requested.



In [8]:
df = pd.read_csv('/content/stud.csv')
display(df.head())

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


## Preprocessing

### Subtask:
Handle categorical features using one-hot encoding and apply standard scaling to numerical features.


**Reasoning**:
Identify categorical and numerical columns, create a ColumnTransformer with one-hot encoding for categorical features and standard scaling for numerical features, and apply this transformation to the DataFrame.



In [9]:
numerical_features = df.select_dtypes(include=np.number).columns
categorical_features = df.select_dtypes(include=np.object_).columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(), categorical_features),
        ('StandardScaler', StandardScaler(), numerical_features)
    ]
)

preprocessed_data = preprocessor.fit_transform(df)
display(preprocessed_data)

array([[ 1.        ,  0.        ,  0.        , ...,  0.39002351,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.19207553,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.57771141,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ..., -0.46775108,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.12609287,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  0.71993682,
         1.15336989,  1.18158627]])

## Split data

### Subtask:
Split the preprocessed data into training and testing sets (80/20 split).


**Reasoning**:
Split the preprocessed data into training and testing sets using train_test_split.



In [10]:
from sklearn.model_selection import train_test_split

X = preprocessed_data[:, :-1]
y = preprocessed_data[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (800, 19)
Shape of X_test: (200, 19)
Shape of y_train: (800,)
Shape of y_test: (200,)


## Model training

### Subtask:
Train various regression models including Linear Regression, Lasso, Ridge, Random Forest, Decision Tree, XGBoost, CatBoost, and KNN.


**Reasoning**:
Instantiate and train the specified regression models using the training data.



In [11]:
# Instantiate Regression models
lr = LinearRegression()
lasso = Lasso()
ridge = Ridge()
rf = RandomForestRegressor()
dt = DecisionTreeRegressor()
xgb = xgb.XGBRegressor()
cat = CatBoostRegressor(verbose=0)  # Set verbose to 0 to suppress output
knn = KNeighborsRegressor()

# Train models
lr.fit(X_train, y_train)
lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)
rf.fit(X_train, y_train)
dt.fit(X_train, y_train)
xgb.fit(X_train, y_train)
cat.fit(X_train, y_train)
knn.fit(X_train, y_train)

print("All models trained successfully.")

All models trained successfully.


## Model evaluation

### Subtask:
Evaluate each model on both the training and testing sets using metrics such as Mean Squared Error (MSE), Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), and R-squared (R2).


**Reasoning**:
Create a function to evaluate model performance using MSE, MAE, RMSE, and R2, then evaluate each trained model on both the training and testing datasets using this function and store the results.



In [12]:
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return {'MSE': mse, 'MAE': mae, 'RMSE': rmse, 'R2': r2}

models = {
    'Linear Regression': lr,
    'Lasso': lasso,
    'Ridge': ridge,
    'Random Forest': rf,
    'Decision Tree': dt,
    'XGBoost': xgb,
    'CatBoost': cat,
    'KNN': knn
}

model_performance = {}

for model_name, model in models.items():
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_metrics = evaluate_model(y_train, y_train_pred)
    test_metrics = evaluate_model(y_test, y_test_pred)

    model_performance[model_name] = {
        'train': train_metrics,
        'test': test_metrics
    }

for model_name, performance in model_performance.items():
    print(f"Model: {model_name}")
    print("Train Metrics:", performance['train'])
    print("Test Metrics:", performance['test'])
    print("-" * 30)

Model: Linear Regression
Train Metrics: {'MSE': 0.049348135252007756, 'MAE': 0.1784229833940632, 'RMSE': np.float64(0.22214440180208853), 'R2': 0.9499149534431213}
Test Metrics: {'MSE': 0.06464023724717562, 'MAE': 0.21070979603971832, 'RMSE': np.float64(0.2542444438865393), 'R2': 0.9381328994057507}
------------------------------
Model: Lasso
Train Metrics: {'MSE': 0.9852868000425218, 'MAE': 0.8022503980463117, 'RMSE': np.float64(0.9926161393220049), 'R2': 0.0}
Test Metrics: {'MSE': 1.0623599766091198, 'MAE': 0.8043655467716306, 'RMSE': np.float64(1.030708482845232), 'R2': -0.01678357535812558}
------------------------------
Model: Ridge
Train Metrics: {'MSE': 0.04934950162387125, 'MAE': 0.17844327024395995, 'RMSE': np.float64(0.22214747719447828), 'R2': 0.9499135666673486}
Test Metrics: {'MSE': 0.06462999602748841, 'MAE': 0.21072725031873468, 'RMSE': np.float64(0.2542243025902292), 'R2': 0.9381427012659477}
------------------------------
Model: Random Forest
Train Metrics: {'MSE': 0.0

## Performance summary

### Subtask:
Create a DataFrame to summarize the performance of each model, including the calculated metrics and adjusted R-squared.


**Reasoning**:
Create a DataFrame to summarize the performance of each model, including the calculated metrics and adjusted R-squared.



In [13]:
model_summary = []

for model_name, performance in model_performance.items():
    train_metrics = performance['train']
    test_metrics = performance['test']

    # Calculate adjusted R-squared for train set
    n_train = len(y_train)
    p_train = X_train.shape[1]
    adjusted_r2_train = 1 - (1 - train_metrics['R2']) * (n_train - 1) / (n_train - p_train - 1)

    # Calculate adjusted R-squared for test set
    n_test = len(y_test)
    p_test = X_test.shape[1]
    adjusted_r2_test = 1 - (1 - test_metrics['R2']) * (n_test - 1) / (n_test - p_test - 1)

    model_summary.append({
        'Model': model_name,
        'Train MSE': train_metrics['MSE'],
        'Train MAE': train_metrics['MAE'],
        'Train RMSE': train_metrics['RMSE'],
        'Train R2': train_metrics['R2'],
        'Train Adjusted R2': adjusted_r2_train,
        'Test MSE': test_metrics['MSE'],
        'Test MAE': test_metrics['MAE'],
        'Test RMSE': test_metrics['RMSE'],
        'Test R2': test_metrics['R2'],
        'Test Adjusted R2': adjusted_r2_test
    })

model_performance_df = pd.DataFrame(model_summary)
display(model_performance_df)

,Model,Train MSE,Train MAE,Train RMSE,Train R2,Train Adjusted R2,Test MSE,Test MAE,Test RMSE,Test R2,Test Adjusted R2
0,Linear Regression,0.049348,0.178423,0.222144,0.949915,0.948695,0.064640,0.210710,0.254244,0.938133,0.931602
1,Lasso,0.985287,0.802250,0.992616,0.000000,-0.024359,1.062360,0.804366,1.030708,-0.016784,-0.124111
2,Ridge,0.049350,0.178443,0.222147,0.949914,0.948694,0.064630,0.210727,0.254224,0.938143,0.931613
3,Random Forest,0.009552,0.078312,0.097732,0.990306,0.990070,0.090445,0.241584,0.300741,0.913435,0.904298
4,Decision Tree,0.000266,0.001317,0.016295,0.999731,0.999724,0.151142,0.307808,0.388770,0.855342,0.840073
5,XGBoost,0.001128,0.021681,0.033581,0.998855,0.998828,0.087257,0.233764,0.295393,0.916486,0.907671
6,CatBoost,0.014442,0.094091,0.120177,0.985342,0.984985,0.086935,0.230189,0.294847,0.916795,0.908012
7,KNN,0.073754,0.218840,0.271578,0.925144,0.923321,0.124850,0.274558,0.353341,0.880506,0.867893


## Model selection

### Subtask:
Select the top 2 best performing models based on the evaluation metrics.


**Reasoning**:
Sort the model performance DataFrame by the 'Test R2' column in descending order and select the top 2 rows to identify the best performing models on the test set. Then extract and print the names of these top 2 models.



In [14]:
top_2_models_df = model_performance_df.sort_values(by='Test R2', ascending=False).head(2)
top_2_model_names = top_2_models_df['Model'].tolist()

print("Top 2 best performing models based on Test R2:")
for model_name in top_2_model_names:
    print(model_name)

Top 2 best performing models based on Test R2:
Ridge
Linear Regression


In [15]:
# Make predictions using the top 2 models
ridge_predictions = ridge.predict(X_test)
lr_predictions = lr.predict(X_test)

print("Predictions from Ridge Model:")
display(ridge_predictions)

print("\nPredictions from Linear Regression Model:")
display(lr_predictions)

Predictions from Ridge Model:


array([ 1.35475126, -0.09421413,  0.3629119 ,  0.2103423 ,  0.77813796,
        0.35549177,  0.02229395, -0.34044387,  0.30829078, -1.08384498,
       -1.77046814, -3.0221014 ,  0.70415824, -0.32689672,  0.92611549,
        0.72178591, -1.24352218, -1.30638509, -0.69775912, -0.17166539,
        0.21571813, -0.78253492,  0.09591386, -1.24410142,  0.64374524,
        0.36144616,  0.40646425, -0.78854748, -1.32373775, -0.6287233 ,
       -0.7266607 , -0.20502159, -0.55703167, -0.12367821,  0.28861625,
       -1.03914838,  0.40805304,  0.59963781,  0.74321694, -3.48842321,
        0.55450157, -0.29765256, -0.17326906, -0.42460598,  1.04569297,
       -0.25425624, -0.08938618, -2.38817047,  1.22974483,  0.95455774,
        0.28058987,  0.48137211,  0.64853555, -0.62138617,  0.27602096,
        0.47028356,  0.73208827, -1.04254926,  0.80897301,  1.49055085,
       -1.75494653,  0.99437693,  0.77470606, -0.44462653,  1.32653163,
        0.65074061,  0.04700124, -1.18652156, -0.00953546,  1.64


Predictions from Linear Regression Model:


array([ 1.35504363, -0.09364176,  0.3637806 ,  0.21049694,  0.77764888,
        0.35564629,  0.02245382, -0.34094202,  0.30756404, -1.08488602,
       -1.77199591, -3.02601086,  0.70500414, -0.3261444 ,  0.92690956,
        0.72265874, -1.24442086, -1.30673231, -0.69816017, -0.17119493,
        0.21674189, -0.78199417,  0.09738209, -1.2445605 ,  0.64396196,
        0.36165484,  0.40825365, -0.7883323 , -1.3243088 , -0.62856461,
       -0.72728673, -0.20531625, -0.5566474 , -0.12515318,  0.28730304,
       -1.0396467 ,  0.40863551,  0.6005509 ,  0.74508924, -3.49068099,
        0.55448742, -0.2982833 , -0.17253874, -0.42477076,  1.0464747 ,
       -0.25558381, -0.0896802 , -2.39035041,  1.23034511,  0.95510213,
        0.28107596,  0.48243913,  0.64896895, -0.62144494,  0.27552959,
        0.47094224,  0.73303394, -1.04361841,  0.80931955,  1.49103233,
       -1.75700796,  0.99503617,  0.77466902, -0.44501039,  1.32953603,
        0.65223406,  0.04800209, -1.18805787, -0.0090279 ,  1.64

## Summary:

### Data Analysis Key Findings

*   The dataset was successfully loaded and preprocessed using one-hot encoding for categorical features and standard scaling for numerical features.
*   The data was split into training (80%) and testing (20%) sets.
*   Eight different regression models (Linear Regression, Lasso, Ridge, Random Forest, Decision Tree, XGBoost, CatBoost, and KNN) were trained on the training data.
*   All models were evaluated on both training and testing sets using MSE, MAE, RMSE, and R2 metrics.
*   A summary DataFrame was created to consolidate the performance metrics for all models on both training and testing sets, including calculated adjusted R-squared values.
*   Based on the 'Test R2' score, the top 2 best performing models were identified as Ridge and Linear Regression.

### Insights or Next Steps

*   Ridge and Linear Regression models show promising performance on the test set, indicating good generalization. Further investigation into potential collinearity or regularization effects might be beneficial.
*   Compare the performance of the top models on other metrics like RMSE and MAE to get a more complete picture of their predictive accuracy.
